In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib widget

In [ ]:
# PLot and global settings
sns.set_style("whitegrid")
sns.set_context("talk")
fig_height = 5
num_bootstraps = 10
font_size_txt_on_plot = 16
font_size_asterisk_on_plot = 20
image_ext = ".eps"

In [ ]:
# plots mean AUC +/- std for each model

def plot_performance(
    model_names: list,
    aucs: dict,
    pvals: list,
    y_min: float = 0.7,
    y_max: float = 0.95,
    x_label_strs: list = [],
    plot_title: str = None,
    fpath: str = None,
    palette = None):
    
    if palette is None:
        palette = sns.color_palette("pastel")
    
    y_delta = 0.1
    
    stats = {model: {'aucs': [], 'mean': 0, 'std': 0} for model in model_names}

    means = [np.nanmean(aucs[model]) for model in model_names]
    stds = [np.nanstd(aucs[model]) for model in model_names]
        
    fig_width = 1 + len(model_names) * 1
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    x_pos = np.arange(len(model_names))
    
    bar = sns.barplot(x=x_pos, y=means, yerr=stds, palette=palette)
    ax.set_ylabel('AUC')
    ax.set_ylim([y_min, y_max])
    ax.set_yticks(np.arange(y_min, y_max+0.01, y_delta))
    
    ax.set_xticks(range(len(model_names)))
    ax.set_xticklabels(model_names if len(x_label_strs) == 0 else x_label_strs)

    if plot_title is not None:
        ax.set_title(plot_title)
    
    # Indicate significance for each column
    for i, patch in enumerate(ax.patches):
        y_height = means[i] + stds[i]
        sig_txt = "*" if pvals[i] < 0.05 else None
        ax.text(patch.get_x() + patch.get_width() / 2,
                means[i] + stds[i],
                s=sig_txt,
                ha='center',
                fontsize=font_size_asterisk_on_plot,
        )
    
    offset = -0.25
    for i, v in enumerate(means):
        plt.text(
            x_pos[i]+offset,
            y_min + 0.01,
            f'{v:0.2f}',
            fontsize=font_size_txt_on_plot,
        )
    plt.xticks(rotation=90)
    plt.tight_layout()
    if fpath is not None:
        plt.savefig(fpath, dpi=300)

In [ ]:
def get_aucs_from_column(df: pd.DataFrame, col_name: str, bootstraps: int = 10) -> list:
    """Given a dataframe and model name (col_name),
    extracts the AUCs and casts to list of floats"""
    aucs = []
    if col_name not in df:
        return aucs
    for bootstrap in range(bootstraps):
        try:
            auc = df[col_name].loc[str(bootstrap)]
            aucs.append(float(auc))
        except:
            aucs.append(np.nan)
            print(f"no valid auc found at bootstrap {bootstrap}")
    return aucs

## Load CSVs and concatenate horizontally into one wide dataframe

In [ ]:
csv_filenames = ['stsnet', 'sts-shallow']
rootdir = os.path.expanduser("~/dropbox/sts-ecg/figures-and-tables")
df = pd.DataFrame()
for csv_filename in csv_filenames:
    fpath = os.path.join(rootdir, f"{csv_filename}.csv")
    df_ = pd.read_csv(fpath, low_memory=False, index_col=0)
    if np.any(["sts" not in col for col in df_.columns]):
        cols = [f"{csv_filename}_{col}" for col in df_.columns]
        df_.columns = cols
    df = pd.concat([df, df_], axis=1)

df.rename(columns={'Unnamed: 0':'parameter'}, inplace=True )

## Get AUCs from dataframe as a dict (keyed by model name) of lists of floats

In [ ]:
aucs = {}
for model in df.keys():
    aucs[model] = get_aucs_from_column(df=df, col_name=model, bootstraps=10)

## Get AUCs of published STS risk models and append to dict of AUCs

In [ ]:
sts = pd.read_csv(os.path.join(rootdir, "sts.csv"))
sts.rename(columns={'Unnamed: 0':'outcome'}, inplace=True )
for cohort in [col for col in sts.columns if col != "outcome"]:
    for outcome in sts.outcome:
        auc = sts[sts.outcome == outcome][cohort]
        aucs[f"sts {outcome} {cohort}"] = [float(auc) for i in range(num_bootstraps)]

In [ ]:
root = os.path.expanduser("~/dropbox/sts-ecg/figures-and-tables")
dirpath = os.path.join(root, "auc-bar-plots")
if not os.path.isdir(dirpath):
    os.mkdir(dirpath)

In [ ]:
root = os.path.expanduser("~/dropbox/sts-ecg/figures-and-tables")

outcomes = [
    "death",
    "stroke",
    "renal",
    "vent",
    "reop",
    "stay",
    "dsw",
]

cohorts = [
    "all",
    "cabg",
    "valve",
    "cabg-valve",
    "others",
    "office",
]

for outcome in outcomes:
    for cohort in cohorts:
        if (cohort == "all") or (cohort == "others") or (cohort == "office"):
            models_to_plot = [
                f'sts-shallow-v001 {outcome} infer {cohort}', 
                f'stsnet-v046 {outcome} infer {cohort}',
            ]
            
            model_1 = f'sts-shallow-v001 {outcome} infer {cohort}'
            model_2 = f'stsnet-v046 {outcome} infer {cohort}'

            _, pval = stats.ttest_rel(
                [auc for auc in aucs[model_1] if auc == auc],
                [auc for auc in aucs[model_2] if auc == auc]
            )

            pvals = np.full(len(models_to_plot), 1.0)      
            pvals[1] = pval
            
            # Remove the first color from the palette because we are not plotting baseline STS AUC
            palette = sns.color_palette("pastel")[1:]
            
            plot_performance(
                model_names=models_to_plot,
                aucs=aucs,
                pvals=pvals,
                y_min=0.5,
                y_max=1.0,
                x_label_strs=[
                    "LogReg (single task)",
                    "STSNet (multitask)",
                ],
                plot_title=f"{outcome} {cohort}",
                fpath=os.path.join(dirpath, f"aucs-{outcome}-{cohort}{image_ext}"),
                palette=palette,
            )
        else:
            models_to_plot = [
                f'sts {outcome} {cohort}',
                f'sts-shallow-v001 {outcome} infer {cohort}', 
                f'stsnet-v046 {outcome} infer {cohort}',
            ]
            
            model_1 = f'sts-shallow-v001 {outcome} infer {cohort}'
            model_2 = f'stsnet-v046 {outcome} infer {cohort}'
            
            _, pval = stats.ttest_rel(
                [auc for auc in aucs[model_1] if auc == auc],
                [auc for auc in aucs[model_2] if auc == auc]
            )

            pvals = np.full(len(models_to_plot), 1.0)
            pvals[2] = pval
            
            plot_performance(
                model_names=models_to_plot,
                aucs=aucs,
                pvals=pvals,
                y_min=0.5,
                y_max=1.0,
                x_label_strs=[
                    "STS",
                    "LogReg (single task)",
                    "STSNet (multitask)",
                ],
                plot_title=f"{outcome} {cohort}",
                fpath=os.path.join(dirpath, f"aucs-{outcome}-{cohort}{image_ext}")
            )